In [1]:
import tensorflow as tf
import tensorflow.keras
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from tensorflow.keras import layers
from tensorflow.keras.layers import (Dropout,Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.utils import layer_utils
import pydot
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

C:\ProgramData\Anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
!pip install pydub

In [3]:
genres = 'blues classical country disco pop hiphop metal reggae rock'
genres = genres.split()
#for g in genres:
 # path_audio = os.path.join('C:/Users/ADMIN/Desktop/Blockchain/dataset/Data/audio3sec',f'{g}')
  #os.makedirs(path_audio)
 # path_train = os.path.join('C:/Users/ADMIN/Desktop/Blockchain/dataset/Data/spectrograms3sec/train',f'{g}')
 # path_test = os.path.join('C:/Users/ADMIN/Desktop/Blockchain/dataset/Data/spectrograms3sec/test',f'{g}')
  #os. makedirs(path_train)
  #os. makedirs(path_test)

In [4]:
train_dir = "C:/Users/ZIshnu/Desktop/spectrograms3sec/test"
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(288,432),color_mode="rgba",class_mode='categorical',batch_size=128)

validation_dir = "C:/Users/ZIshnu/Desktop/spectrograms3sec/train"
vali_datagen = ImageDataGenerator(rescale=1./255)
vali_generator = vali_datagen.flow_from_directory(validation_dir,target_size=(288,432),color_mode='rgba',class_mode='categorical',batch_size=128)


Found 9000 images belonging to 9 classes.
Found 9000 images belonging to 9 classes.


In [5]:
def GenreModel(input_shape = (288,432,4),classes=9):
  
  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)
  
  X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model

In [12]:
import tensorflow.keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
  
model = GenreModel(input_shape=(288,432,4),classes=9)
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy',get_f1]) 

history = model.fit_generator(train_generator,epochs=20,validation_data=vali_generator)

C:\Users\ZIshnu\AppData\Local\Temp/ipykernel_35004/2916766418.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,epochs=20,validation_data=vali_generator)


Epoch 1/20
71/71 [==============================] - 324s 5s/step - loss: 1.4542 - accuracy: 0.5233 - get_f1: 0.4798 - val_loss: 2.2531 - val_accuracy: 0.2217 - val_get_f1: 0.1834
Epoch 2/20
71/71 [==============================] - 325s 5s/step - loss: 0.7010 - accuracy: 0.7450 - get_f1: 0.7293 - val_loss: 2.9842 - val_accuracy: 0.2222 - val_get_f1: 0.2231
Epoch 3/20
71/71 [==============================] - 321s 5s/step - loss: 0.5143 - accuracy: 0.8188 - get_f1: 0.8133 - val_loss: 3.7128 - val_accuracy: 0.2256 - val_get_f1: 0.2238
Epoch 4/20
71/71 [==============================] - 322s 5s/step - loss: 0.4098 - accuracy: 0.8537 - get_f1: 0.8523 - val_loss: 3.4830 - val_accuracy: 0.2608 - val_get_f1: 0.2594
Epoch 5/20
71/71 [==============================] - 322s 5s/step - loss: 0.3325 - accuracy: 0.8837 - get_f1: 0.8812 - val_loss: 1.1310 - val_accuracy: 0.5774 - val_get_f1: 0.5627
Epoch 6/20
71/71 [==============================] - 325s 5s/step - loss: 0.2539 - accuracy: 0.9162 - get_

In [8]:
model.summary()

Model: "GenreModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 288, 432, 4)]     0         
                                                                 
 conv2d (Conv2D)             (None, 286, 430, 8)       296       
                                                                 
 batch_normalization (BatchN  (None, 286, 430, 8)      32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 286, 430, 8)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 143, 215, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 141, 213, 16)      1

In [13]:
model.save('aiproject.model')

INFO:tensorflow:Assets written to: aiproject.model\assets
